## 11.1 교차검증 모델 만들기
실전에서 모델이 얼마나 잘 작동할지 평가하고 싶습니다.

데이터 전처리 파이프라인을 만들고 모델을 훈련한 다음 교차검증으로 평가합니다.

In [4]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# 숫자 데이터셋을 로드합니다.
digits = datasets.load_digits()

# 특성 행렬을 만듭니다.
features = digits.data

# 타깃 벡터를 만듭니다.
target = digits.target

# 표준화 객체를 만듭니다.
standardizer = StandardScaler()

# 로지스틱 회귀 객체를 만듭니다.
logit = LogisticRegression()

# 표준화한 다음 로지스틱 회귀를 실행하는 파이프라인을 만듭니다.
pipeline = make_pipeline(standardizer, logit)

# k-폴드 교차검증을 만듭니다.
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# k-폴드 교차검증을 수행합니다.
cv_results = cross_val_score(pipeline, # 파이프라인
                             features, # 특성행렬
                             target, # 타깃 벡터
                             cv=kf, # 교차검증 기법
                             scoring="accuracy", # 평가 지표
                             n_jobs=-1) # 모든 CPU 코어 사용

# 평균을 계산합니다.
cv_results.mean()

0.9693916821849783

k-폴드 교차검증(k-fold cross-validation, KFCV)에서는 데이터를 fold라고 부르는 $k$개의 부분으로 나눕니다.$k$-1개 폴드를 하나의 훈련 세트로 합쳐 모델을 훈련하고 남은 폴드를 테스트 세트처럼 사용합니다. 이를 $k$번 반복합니다. 반복마다 다른 폴드를 테스트 세트로 사용합니다. $k$번 반복에서 얻은 모델 성능을 평균하여 최종 성능을 산출합니다.

해결에서 10개의 폴드를 사용하여 $k$-폴드 교차검증을 수행했습니다. 평가 정수는 cv_results에 저장되어 있습니다.

In [5]:
# 10개 폴드의 점수를 모두 확인하기
cv_results

array([0.97777778, 0.98888889, 0.96111111, 0.94444444, 0.97777778,
       0.98333333, 0.95555556, 0.98882682, 0.97765363, 0.93854749])

KFCV를 사용할 때 고려해야 할 중요한 점이 세 가지 있습니다. 첫째, KFCV는 각 샘플이 다른 샘플과 독립적으로 생성되었다고 가정합니다(즉 데이터는 independent identically distributed, iid입니다). 데이터가 iid라면 폴드를 나누기 전에 샘플을 섞는 것이 좋은 생각입니다. 사이킷런에서는 shuffle=True로 지정하여 섞을 수 있습니다.

둘째, KFCV를 사용하여 classifier를 평가할 때, 각 타깃 클래스의 샘플이 거의 같은 비율로 폴드에 담기는 것이 좋습니다(계층별 $k$-폴드, stratified k-fold라고 부릅니다). 예를 들어 성별 타깃 벡터 중에서 80% 샘플이 남성이라면 각 폴드도 80% 남성과 20% 여성 샘플로 이루어져야 합니다. 사이킷런에서는 KFold 클래스를 StratifiedKFold로 바꾸어 계층별 $k$-폴드 교차검증을 수행할 수 있습니다.

마지막으로 검증 세트나 교차검증을 사용할 때 훈련 세트에서 데이터를 전처리하고 이 변환을 훈련 세트와 테스트 세트에 모두 적용하는 것이 중요합니다. 예를 들면 표준화 객체 standardizer의 fit 메서드를 호출하여 훈련 세트의 평균과 분산을 계산합니다. 그다음 이 변환을 (transform 메서드를 사용해) 훈련 세트와 테스트 세트에 모두 적용합니다.

In [7]:
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.1, random_state=1)

# 훈련 세트로 standardizer의 fit 메서드를 호출합니다.
standardizer.fit(features_train)

# 훈련 세트와 테스트 세트에 모두 적용합니다.
features_train_std = standardizer.transform(features_train)
features_test_std = standardizer.transform(features_test)

이렇게 하는 이유는 테스트 세트를 모르는 척하기 위해서입니다. 이 전처리 객체를 훈련 세트와 테스트 세트에 있는 모든 샘플로 훈련한다면 테스트 세트의 정보가 훈련 세트로 유출된 것입니다. 이 규칙은 특성 선택 같은 모든 전처리 단계에 적용됩니다.

사이킷런의 pipeline 패키지는 교차검증 기법을 사용할 때 이 규칙을 손쉽게 구현할 수 있도록 도와줍니다. 먼저 데이터를 전처리 (예를 들면 standardizer)하고 모델(로지스틱 회귀인 logit)을 훈련하는 파이프라인을 만듭니다.

In [8]:
# 파이프라인을 만듭니다.
pipeline = make_pipeline(standardizer, logit)

그다음 이 파이프라인으로 KFCV를 실행하면 사이킷런이 모든 작업을 알아서 처리합니다.

In [9]:
# k-폴드 교차검증 수행
cv_results = cross_val_score(pipeline, # 파이프라인
                             features, # 특성행렬
                             target, # 타깃 벡터
                             cv=kf, # 교차검증 기법
                             scoring="accuracy", # 평가 지표
                             n_jobs=-1) # 모든 CPU 코어 사용

cross_val_score에는 아직 이야기하지 않은 중요한 세 개의 매개변수가 있습니다. cv는 교차검증 기법을 결정합니다. $k$-폴드를 가장 많이 사용하지만 다른 방식도 있습니다. LOOCV(leave-one-out-cross-validation)는 폴드의 수 $k$가 샘플의 개수와 같습니다. scoring 매개변수는 이 장의 다른 여러 레시피에서 설명할 모델 성공의 측정 방법을 결정합니다. 마지막으로 n_jobs=-1은 사이킷런에게 가용한 모든 코어를 사용하도록 지시합니다. 예를 들어 사용하는 컴퓨터에 (요즘 노트북의 기본 사양인) 네 개의 코어가 있다면 사이킷런은 네 개의 코어를 모두 동시에 사용해 작업의 속도를 높입니다.

LOOCV는 LeaveOneOut 클래스에 구현되어 있습니다. LeaveOneOut 클래스는 KFold(n_splits=n)과 동일합니다(n은 샘플 개수).

KFold와 StratifiedKFold의 n_splits 매개변수 기본값은 3입니다. 사이킷런 0.22 버전부터 이 값이 5로 바뀝니다.

ShuffleSplit는 반복 횟수에 상관없이 훈련 폴드와 테스트 폴드 크기를 임의로 지정할 수 있습니다. train_size, test_size 매개변수에는 사용할 샘플 개수 또는 비율을 입력합니다. 반복마다 랜덤하게 분할하기 때문에 하나의 샘플이 여러 번 테스트 폴드에 포함될 수 있습니다. 계층별 교차검증을 위한 StratifiedShuffleSplit도 있습니다. 다음 코드는 훈련 폴드로 50%, 테스트 폴드로 20%를 사용하여 10번 반복하는 예입니다.

In [10]:
from sklearn.model_selection import ShuffleSplit

# ShuffleSplit 분할기를 만듭니다.
ss = ShuffleSplit(n_splits=10, train_size=0.5, test_size=0.2, random_state=42)

# 교차검증을 수행합니다.
cv_results = cross_val_score(pipeline,
                             features,
                             target,
                             cv=ss,
                             scoring="accuracy",
                             n_jobs=-1)

# 평균을 계산합니다.
cv_results.mean()

0.9630555555555554

사이킷런 0.19 버전에서는 교차검증을 반복하여 실행할 수 있는 RepeatedKFold와 StratifiedRepeatedKFold가 추가되었습니다. 다음 코드는 10-폴드 교차검증을 5번 반복하는 RepeatedKFold의 예입니다.

In [11]:
from sklearn.model_selection import RepeatedKFold

# RepeatedKFold 분할기를 만듭니다.
rfk = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)

# 교차검증을 수행합니다.
cv_results = cross_val_score(pipeline,
                             features,
                             target,
                             cv=rfk,
                             scoring="accuracy",
                             n_jobs=-1)

# 검증 점수 개수를 확인합니다.
len(cv_results)

50

총 50개의 교차검증 점수가 생성되었습니다. n_splits 매개변수 기본값은 5이고 n_repeats 매개변수 기본값은 10입니다.